<strong>Dependencies</strong>: MeshPy, PyVTK, trimesh

<strong>Useful links</strong>:
 - TetGen : http://wias-berlin.de/software/tetgen/1.5/doc/manual/manual006.html
 - MeshPy : https://documen.tician.de/meshpy/tri-tet.html#meshpy.tet.MeshInfo.adjacent_elements
 
<strong>Useful research</strong>:
 - SRF BluePrint : http://i.cs.hku.hk/~wenping/allhex.pdf
 - QuadCover : 
 - CubeCover : http://tinyurl.com/zd7yj6j
 - Principal Curvature : http://gfx.cs.princeton.edu/pubs/_2004_ECA/curvpaper.pdf

In [1]:
import meshpy.tet
import numpy as np
import trimesh

## Generate Tetrahedral Mesh

In [7]:
tri_mesh = trimesh.load_mesh('../data/cylinder.stl')

# Define MeshPy options!
opt = meshpy.tet.Options(switches='pq', edgesout=True, facesout=True, neighout=True)

mesh_info = meshpy.tet.MeshInfo()
mesh_info.set_points(tri_mesh.vertices)
# Convert face data from np.int64 to int...
faces = [list(map(lambda x: int(x), i)) for i in tri_mesh.faces]
mesh_info.set_facets(faces)
tet_mesh = meshpy.tet.build(mesh_info, opt, max_volume=10)

# Output tetrahedral mesh
tet_mesh.write_vtk("../data/test.vtk")

## Compute surface normals

#### Some useful information

 - Internal tetrahedrons have 4 neighbors (one on each face) - the neighbor 4-tuple is the indices of the neighboring tets.

 - The first neighbor of tetrahedron 'i' is oppposite to the first corner of tetrahedron 'i', and so on.

 - An index of -1 indicates there is no neighbor.
 
 - Tetrahedral face orientation:
 
    - 3 - 0 - 1
 
    - 1 - 2 - 3
   
    - 3 - 2 - 0  (unusual)
   
    - 2 - 1 - 0  (unusual)
   

In [8]:
surface_faces = []
for i, tet in enumerate(tet_mesh.elements):
    neighbors = list(tet_mesh.neighbors[i])
    # TODO(aidan) This overlooks case of multiple surface faces...
    try:
        non_surface_vtx = tet[neighbors.index(-1)]
        tet_cpy = tet.copy()
        tet_cpy.remove(non_surface_vtx)
        surface_faces.append(tet_cpy)
    except ValueError:
        continue
        
# Compute normal
def compute_normal(face):
    return np.cross(np.array(tet_mesh.points[face[0]]) - np.array(tet_mesh.points[face[1]]),
                    np.array(tet_mesh.points[face[2]]) - np.array(tet_mesh.points[face[1]]))

def compute_avg(face):
    return (np.array(tet_mesh.points[face[0]])
          + np.array(tet_mesh.points[face[1]])
          + np.array(tet_mesh.points[face[2]])) / 3

face_normals = [compute_normal(face) for face in surface_faces]
face_center = [compute_avg(face) for face in surface_faces]

### Visualize normals

In [9]:
from mpl_toolkits.mplot3d import proj3d
import mpl_toolkits.mplot3d as a3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(num=None, figsize=(12, 10), dpi=80)
ax = fig.add_subplot(111, projection='3d')

for i, f in enumerate(surface_faces):
    ax.quiver(face_center[i][0], face_center[i][1], face_center[i][2], face_normals[i][0], face_normals[i][1], face_normals[i][2])

plt.show()

## Generate SRF

## Plot Struts

KeyboardInterrupt: 